# Obtaining data

In this notebook, we're going to load a few different datasets, inspect them, and do any necessary cleaning so that we have datasets that we can measure the effectiveness of KNN Imputation against.

Our datasets will include:

* The Iris Dataset
* The Titanic Dataset 
* The King County Housing Dataset
* The Terry Stops Dataset
* 

Let's load our necessary libraries.

# Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Inspecting and Scrubbing

## Iris Dataset

The Iris Dataset is a perfectly clean clean and curated dataset.  The features are all continuous variables while the target is divided into 3 separate classes.  The data is evenly balanced and the target variable even comes out of the box with numerical dummy labels.  We're going to take a step back and relabel the target classes for future interpretability.

In [7]:
from sklearn import datasets

# loading dataset
data = datasets.load_iris()

# Concatenating data into Dataframe
# Using Numpy's Concatenate function to join (np.c_) to join the datasets 
iris_df = pd.DataFrame(data=np.c_[data['data'], data['target']], 
                                  columns=data['feature_names'] + ['target'])
iris_df['target'] = iris_df.target.astype(int)
iris_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [17]:
# Returning labels to classes
class_ = ['0', '1', '2']
label = ['setosa', 'versicolor', 'virginica']
iris_df['target'] = iris_df.target.astype(str)
for i in range(len(iris_df)):
    for j in range(len(class_)):
        if iris_df['target'][i] == '0':
            iris_df['target'][i] = iris_df['target'][i].replace(class_[j], 
                                                                'setosa')
        elif iris_df['target'][i] == '1':
            iris_df['target'][i] = iris_df['target'][i].replace(class_[j], 
                                                                label[j])
        elif iris_df['target'][i] == '2':
            iris_df['target'][i] = iris_df['target'][i].replace(class_[j], 
                                                                label[j])
iris_df.target.value_counts()
            

C:\Users\User\anaconda3\envs\learn-env\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\Users\User\anaconda3\envs\learn-env\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
C:\Users\User\anaconda3\envs\learn-env\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from

setosa        50
virginica     50
versicolor    50
Name: target, dtype: int64

In [18]:
iris_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [19]:
# exporting to csv
iris_df.to_csv('datasets/iris/iris_cleaned')

## Titanic Dataset
This is a classic which again is used for classification problems.  This will be a nice and easy dive into applying KNN Imputation to slightly larger datasets that contain categorical, continuous, and discreet variables.